In [ ]:
from datetime import datetime
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm

import MetaTrader5 as mt5
import pandas as pd
import numpy as np
import talib
import joblib
import os

In [2]:
mt5_path = "C:\\MT5\\terminal64.exe"

# 連接 MT5
if not mt5.initialize(path=mt5_path, portable = True):
    print(f"{mt5.last_error()}")
    mt5.shutdown()


# 獲取 Market Watch 內的所有交易品種
symbols = mt5.symbols_get()
symbol_list = [symbol.name for symbol in symbols]

print("Market Watch 內的品種數量:", len(symbol_list))
print("品種列表:", symbol_list)

# 斷開連線
mt5.shutdown()

Market Watch 內的品種數量: 166
品種列表: ['XTIUSD.x', 'XAGUSD.x', 'AMD', 'AVGO', 'BA', 'EBAY', 'GE', 'GOOG', 'GOOGL', 'JNJ', 'NFLX', 'RTX', 'TSLA', 'UNH', 'DAX30', 'HSI50', 'NAS100', 'NK225', 'SPX500', 'US30', 'XAUUSD.x', 'XBRUSD.x', 'AUDCAD.x', 'AUDCHF.x', 'AUDJPY.x', 'AUDNZD.x', 'AUDUSD.x', 'CADCHF.x', 'CADJPY.x', 'CHFJPY.x', 'EURAUD.x', 'EURCAD.x', 'EURCHF.x', 'EURGBP.x', 'EURJPY.x', 'EURNZD.x', 'EURUSD.x', 'GBPAUD.x', 'GBPCAD.x', 'GBPCHF.x', 'GBPJPY.x', 'GBPNZD.x', 'GBPUSD.x', 'NZDCAD.x', 'NZDCHF.x', 'NZDJPY.x', 'NZDUSD.x', 'USDCAD.x', 'USDCHF.x', 'USDCNH.x', 'USDHKD.x', 'USDJPY.x', 'USDSEK.x', 'USDSGD.x', 'USDZAR.x', 'AIG', 'APA', 'BRKb', 'EOG', 'FLEX', 'GM', 'JBL', 'LMT', 'SLB', 'UPS', 'QQQ', 'VIXY', 'ABBV', 'ABT', 'CAT', 'CMCSA', 'COST', 'CRM', 'CVX', 'DIS', 'DOW', 'FDX', 'IBM', 'LOW', 'MA', 'MMM', 'MS', 'NEE', 'NKE', 'PYPL', 'TGT', 'V', 'WBA', 'WFC', 'ADBE', 'A', 'AMZN', 'AXP', 'AMGN', 'ADI', 'AAPL', 'AMAT', 'T', 'BAC', 'BBY', 'BIIB', 'BMY', 'BSX', 'CHKP', 'CIEN', 'CRUS', 'CSCO', 'C', 'K

True

In [3]:
# 獲取歷史數據

data = {}
timeframe = mt5.TIMEFRAME_M30
n_candles = 1000

for symbol in symbol_list:
    mt5.initialize()
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, 1000)
    if rates is not None:
        df = pd.DataFrame(rates)
        df['time'] = pd.to_datetime(df['time'], unit='s')
        data[symbol] = df
        print(f"{symbol} 歷史數據獲取成功")
    else:    
        print(f"{symbol} 歷史數據獲取失敗")

    mt5.shutdown()

XTIUSD.x 歷史數據獲取成功
XAGUSD.x 歷史數據獲取成功
AMD 歷史數據獲取成功
AVGO 歷史數據獲取成功
BA 歷史數據獲取成功
EBAY 歷史數據獲取成功
GE 歷史數據獲取成功
GOOG 歷史數據獲取成功
GOOGL 歷史數據獲取成功
JNJ 歷史數據獲取成功
NFLX 歷史數據獲取成功
RTX 歷史數據獲取成功
TSLA 歷史數據獲取成功
UNH 歷史數據獲取成功
DAX30 歷史數據獲取成功
HSI50 歷史數據獲取成功
NAS100 歷史數據獲取成功
NK225 歷史數據獲取成功
SPX500 歷史數據獲取成功
US30 歷史數據獲取成功
XAUUSD.x 歷史數據獲取成功
XBRUSD.x 歷史數據獲取成功
AUDCAD.x 歷史數據獲取成功
AUDCHF.x 歷史數據獲取成功
AUDJPY.x 歷史數據獲取成功
AUDNZD.x 歷史數據獲取成功
AUDUSD.x 歷史數據獲取成功
CADCHF.x 歷史數據獲取成功
CADJPY.x 歷史數據獲取成功
CHFJPY.x 歷史數據獲取成功
EURAUD.x 歷史數據獲取成功
EURCAD.x 歷史數據獲取成功
EURCHF.x 歷史數據獲取成功
EURGBP.x 歷史數據獲取成功
EURJPY.x 歷史數據獲取成功
EURNZD.x 歷史數據獲取成功
EURUSD.x 歷史數據獲取成功
GBPAUD.x 歷史數據獲取成功
GBPCAD.x 歷史數據獲取成功
GBPCHF.x 歷史數據獲取成功
GBPJPY.x 歷史數據獲取成功
GBPNZD.x 歷史數據獲取成功
GBPUSD.x 歷史數據獲取成功
NZDCAD.x 歷史數據獲取成功
NZDCHF.x 歷史數據獲取成功
NZDJPY.x 歷史數據獲取成功
NZDUSD.x 歷史數據獲取成功
USDCAD.x 歷史數據獲取成功
USDCHF.x 歷史數據獲取成功
USDCNH.x 歷史數據獲取成功
USDHKD.x 歷史數據獲取成功
USDJPY.x 歷史數據獲取成功
USDSEK.x 歷史數據獲取成功
USDSGD.x 歷史數據獲取成功
USDZAR.x 歷史數據獲取成功
AIG 歷史數據獲取成功
APA 歷史數據獲取成功
BRKb 歷史數據獲取成功
EOG 歷史數據獲取成功
FLEX 歷史數據獲取成功
GM 歷史數據獲取成功
JBL

In [4]:
data['XOM']

,time,open,high,low,close,tick_volume,spread,real_volume
0,2024-11-01 22:30:00,117.371,117.561,116.341,116.851,1056,78,0
1,2024-11-01 23:00:00,116.841,117.331,116.741,116.911,857,78,0
2,2024-11-01 23:30:00,116.921,117.211,116.601,116.751,828,78,0
3,2024-11-02 00:00:00,116.741,117.121,116.561,116.641,938,78,0
4,2024-11-02 00:30:00,116.671,116.751,116.091,116.231,1017,78,0
...,...,...,...,...,...,...,...,...
995,2025-02-27 02:30:00,108.631,108.881,108.631,108.841,393,58,0
996,2025-02-27 03:00:00,108.841,109.061,108.681,108.971,524,58,0
997,2025-02-27 03:30:00,108.981,109.371,108.951,109.081,387,58,0
998,2025-02-27 04:00:00,109.071,109.321,109.071,109.141,445,58,0


In [5]:
# 技術指標
def add_indicator(df):
    df['SMA_10'] = talib.SMA(df['close'], timeperiod=10)
    df['SMA_50'] = talib.SMA(df['close'], timeperiod=50)
    df['RSI'] = talib.RSI(df['close'], timeperiod=14)
    df['ATR'] = talib.ATR(df['high'], df['low'], df['close'], timeperiod=14)
    return df


In [6]:
# 為所有貨幣對添加技術指標
for symbol in data:
    data[symbol] = add_indicator(data[symbol])

# 檢視 EURUSD 的數據
data["XTIUSD.x"].tail()

,time,open,high,low,close,tick_volume,spread,real_volume,SMA_10,SMA_50,RSI,ATR
995,2025-02-27 13:00:00,68.63,68.67,68.62,68.62,39,5,0,68.648,68.7590,46.992170,0.138479
996,2025-02-27 13:30:00,68.62,68.68,68.57,68.65,87,5,0,68.654,68.7528,48.729428,0.136445
997,2025-02-27 14:00:00,68.65,68.72,68.63,68.68,88,5,0,68.656,68.7460,50.477315,0.133127
998,2025-02-27 14:30:00,68.69,68.69,68.62,68.62,85,5,0,68.651,68.7380,47.024421,0.128618
999,2025-02-27 15:00:00,68.62,68.63,68.50,68.50,94,5,0,68.628,68.7266,40.985834,0.128717


In [10]:
os.makedirs("./models", exist_ok=True)
model_path = "./models"

for symbol in tqdm(symbol_list):
    if symbol not in [file_name.replace(".pkl", "") for file_name in os.listdir(model_path)]:
        # 設定訓練數據
        X = data[symbol][['SMA_10', 'SMA_50', 'RSI', 'ATR', 'open', 'high', 'low', 'close', 'tick_volume']].dropna()
        # 預測下一根 K 線的收盤價
        y = data[symbol]['close'].shift(-1).dropna()


        # 對齊 X 和 y
        # 最後一天不會有 y 值，所以要刪除
        X = X.iloc[:-1]
        # 前 48 個數值因為計算SMA50，所以沒有X值要刪除
        y = y.iloc[48:-1]

        # 拆分訓練和測試數據
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # 訓練隨機森林模型
        model = RandomForestRegressor(n_estimators=100, random_state=42)
        model.fit(X_train, y_train)

        # 預測
        y_pred = model.predict(X_test)

        # 模型評估
        mae = mean_absolute_error(y_test, y_pred)

        # 檢視預測結果
        y_test = y_test.reset_index(drop=True)
        y_pred = pd.Series(y_pred)

        result = pd.concat([y_test, y_pred], axis=1)
        result.columns = ['actual', 'predicted']

        # 保存模型
        joblib.dump(model, f"{model_path}/{symbol}")

100%|██████████| 166/166 [01:06<00:00,  2.51it/s]
